## 文档加载

我们使用 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [10]:
## 安装必要的库
# !pip install rapidocr_onnxruntime -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install "unstructured[all-docs]" -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install pyMuPDF -i https://pypi.tuna.tsinghua.edu.cn/simple

In [7]:
from langchain.document_loaders import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../docs/LeeDL-Tutorial/LeeDL_Tutorial.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()

### 1.3 探索加载的数据

文档加载后储存在`pages`变量中:
- `page`的变量类型为`List`
- 打印 `pages` 的长度可以看到pdf一共包含多少页

In [8]:
print(f"载入后的变量类型为：{type(pages)}，",  f"该 PDF 一共包含 {len(pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 330 页


`page`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [24]:
page = pages[8]
print(f"每一个元素的类型：{type(page)}.", 
    f"该文档的描述性数据：{page.metadata}", 
    f"查看该文档的内容：{page.page_content[0:500]}", 
    sep="\n\n")

每一个元素的类型：<class 'langchain.schema.document.Document'>.

该文档的描述性数据：{'source': '../docs/LeeDL-Tutorial/LeeDL_Tutorial.pdf', 'file_path': '../docs/LeeDL-Tutorial/LeeDL_Tutorial.pdf', 'page': 8, 'total_pages': 330, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20230831225119+08'00'", 'modDate': '', 'trapped': ''}

查看该文档的内容：→_→
https://github.com/datawhalechina/leedl-tutorial
←_←
第 1 章
机器学习基础
首先简单介绍一下机器学习（machine learning）和深度学习（deep learning）的基本概
念。机器学习，顾名思义，机器具备有学习的能力。具体来讲，机器学习就是让机器具备找一
个函数的能力。机器具备找函数的能力以后，它可以做很多事，举个例子：
• 语音识别：机器听一段声音，产生这段声音对应的文字。我们需要的是一个函数，该函
数的输入是声音信号，输出是这段声音信号的内容。这个函数显然非常复杂，人类难以
把它写出来，因此想通过机器的力量把这个函数自动找出来。
• 还有好多的任务需要找一个很复杂的函数，以图像识别为例，图像识别函数的输入是一
张图片，输出是这个图片里面的内容。
• AlphaGo 也可以看作是一个函数，机器下围棋需要的就是一个函数，该函数的输入是棋
盘上黑子跟白子的位置。输出是机器下一步应该落子的位置。
随着要找的函数不同，机器学习有不同的类别。假设要找的函数的输出是一个数值，一个
标量（scalar），这种机器学习的任


可以看到我们成功导入了我们所需的文档。接下来我们将文档进行切分

## 三、文档分割方式
Langchain 中文本分割器都根据 chunk_size (块大小)和 chunk_overlap (块与块之间的重叠大小)进行分割。

![image.png](attachment:image.png)

* chunk_size 指每个块包含的字符或 Token（如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

![image.png](attachment:image-2.png)

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小

- RecursiveCharacterTextSplitter():按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter()：按字符来分割文本。
- MarkdownHeaderTextSplitter()：基于指定的标题来分割markdown 文件。
- TokenTextSplitter()：按token来分割文本。
- SentenceTransformersTokenTextSplitter() : 按token来分割文本
- Language() - 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter()：使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter() - 使用 Spacy按句子的切割文本。

### 3.2 基于token的分割
很多 LLM 的上下文窗口长度限制是按照 Token 来计数的。因此，以 LLM 的视角，按照 Token 对文本进行分隔，通常可以得到更好的结果。

In [26]:
# 知识库中单段文本长度
CHUNK_SIZE = 250

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

# 知识库匹配向量数量
VECTOR_SEARCH_TOP_K = 5

In [35]:
# 使用token分割器进行分割，
# 将块大小设为1，块重叠大小设为0，相当于将任意字符串分割成了单个Token组成的列
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)

text_splitter.split_text(page.page_content[0:500])

# loader.load_and_split(text_splitter)

['→_→\nhttps://github.com/datawhalechina/leedl-tutorial\n←_←\n第 1 章\n机器学习基础\n首先简单介绍一下机器学习（machine learning）和深度学习（deep learning）的基本概\n念。机器学习，顾名思义，机器具备有学习的能力。具体来讲，机器学习就是让机器具备找一\n个函数的能力。机器具备找函数的能力以后，它可',
 '以做很多事，举个例子：\n• 语音识别：机器听一段声音，产生这段声音对应的文字。我们需要的是一个函数，该函\n数的输入是声音信号，输出是这段声音信号的内容。这个函数显然非常复杂，人类难以\n把它写出来，因此想通过机器的力量把这个函数自动找出来。\n• 还',
 '有好多的任务需要找一个很复杂的函数，以图像识别为例，图像识别函数的输入是一\n张图片，输出是这个图片里面的内容。\n• AlphaGo 也可以看作是一个函数，机器下围棋需要的就是一个函数，该函数的输入是棋\n盘上黑子跟白子的位置。输出是机器下一步应该落子的位置。\n随着要找的函数不',
 '同，机器学习有不同的类别。假设要找的函数的输出是一个数值，一个\n标量（scalar），这种机器学习的任']

In [42]:
split_docs = text_splitter.split_documents(pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：2236


## 三、Embeddings

什么是`Embeddings`？


在机器学习和自然语言处理（NLP）中，`Embeddings`（嵌入）是一种将类别数据，如单词、句子或者整个文档，转化为实数向量的技术。这些实数向量可以被计算机更好地理解和处理。嵌入背后的主要想法是，相似或相关的对象在嵌入空间中的距离应该很近。

举个例子，我们可以使用词嵌入（word embeddings）来表示文本数据。在词嵌入中，每个单词被转换为一个向量，这个向量捕获了这个单词的语义信息。例如，"king" 和 "queen" 这两个单词在嵌入空间中的位置将会非常接近，因为它们的含义相似。而 "apple" 和 "orange" 也会很接近，因为它们都是水果。而 "king" 和 "apple" 这两个单词在嵌入空间中的距离就会比较远，因为它们的含义不同。

让我们取出我们的切分部分并对它们进行`Embedding`处理。

这里提供两种方式进行，一种是直接使用 openai 的模型去生成 embedding，另一种是使用 HuggingFace 上的模型去生成 embedding。

- openAI 的模型需要消耗 api，对于大量的token 来说成本会比较高，但是非常方便。
- HuggingFace 的模型可以本地部署，可自定义合适的模型，可玩性较高，但对本地的资源有部分要求。


对于只想体验一下的同学来说，可以尝试直接用生成好的 embedding，或者在本地部署小模型进行尝试。

HuggingFace 是一个优秀的开源库，我们只需要输入模型的名字，就会自动帮我们解析对应的能力。

In [ ]:
model_name = "moka-ai/m3e-base"

In [37]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = OpenAIEmbeddings() 
embeddings = HuggingFaceEmbeddings(model_name=model_name)

NameError: name 'HuggingFaceEmbeddings' is not defined

In [45]:
query1 = "什么是机器学习"
query2 = "什么是强化学习"
query2 = "什么是独立学习"

emb1 = embedding.embed_query(query1)
emb2 = embedding.embed_query(query1)
emb3 = embedding.embed_query(query1)

NameError: name 'embedding' is not defined

In [46]:
print(f"{query1} 对应的 embedding： {emb1})

什么是机器学习


通常，通过点积查看两个向量的结果来判断向量之间的相似度。